In [1]:
import pandas as pd
import re

from matminer.utils.conversions import str_to_composition
from matminer.featurizers import composition

In [2]:
df  = pd.read_pickle('../deliver/NIST_CeramicDataSet.pkl')
df.head()

,chemicalFormula,licenses,names,preparation,references,Axis Length,Axis Length-conditions,Axis Length-units,Bulk Modulus,Bulk Modulus-conditions,...,Thermal Expansion-conditions,Thermal Expansion-units,Thermal Shock Resistance,Thermal Shock Resistance-conditions,Thermal Shock Resistance-units,Weibull Modulus,Weibull Modulus-conditions,Weibull Strength,Weibull Strength-conditions,Weibull Strength-units
1,SiO2,[{'name': 'NIST_SRD-30'}],[Silica],NaN,"[{'citation': 'Gaseous Corrosion of Ceramics, ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3Al2O3.2SiO2.xZrO2.yY2O3,[{'name': 'NIST_SRD-30'}],[Mullite composite],[{'name': 'Hot Pressing'}],[{'citation': 'Thermal Diffusivity/Conductivit...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Al2O3,[{'name': 'NIST_SRD-30'}],[Alumina],[{'name': 'Hot Pressing'}],"[{'citation': 'Mechanical Properties of Pure, ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Si3N4.xCeO2.ySiC,[{'name': 'NIST_SRD-30'}],[Silicon nitride composite],[{'name': 'Hot Pressing'}],[{'citation': 'Effect of Silicon Carbide Whisk...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Si3N4,[{'name': 'NIST_SRD-30'}],[Silicon nitride],[{'name': 'Hot Isostatic Pressing'}],[{'citation': 'Corrosion of Silicon Nitride Ce...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Composition parser via regular expression

In [3]:
# Parse the chemicalFormula, v2
# it cannot dealing leading parentheses well yet, such as  (3+x)Al2O3.(2-x)SiO2

def cleanUp(formula): #modified from Chris's function: make_chem_form_compatible
    
    for bad_str in ['whisker', r'\(([a-z]{3,}\-?[a-z]{3,})\)']:    # delete bad strings, (solidsolution), (two-phase)
        formula = re.sub(bad_str, '', formula)
        
    formula = re.sub(r'\$(.*?)\$-', '', formula) # LaTeX expressions
    formula = re.sub(r'Cordierite', 'Mg2Fe2Al4Si5O18', formula) #chemical formular for Cordierite
    formula = re.sub(r'xSialon', 'Si2–xAlxO1+xN2–x', formula) #chemical formular for Sialon
    
    return formula

def formula_decompose(formula):
    '''
    decompose chemical formula 
    return
        composition: list, [(element,num),...]
            element: string
            num: string, can be math expression such as '1+0.5x'
    '''

    comp = []

    formula = cleanUp(formula)
    
    # recognize (prefactor)(elements)(numbers)
    # for example: 2Al2O3 is recognized as 4Al and 6O
    p = re.compile(r'(\d?[w-z]?)([A-Z][a-u]?)(\d*\+?\-?\d*\.?\d*[w-z]?)')

    #split the chemical formula if there is dots, but not for cases like Mg1.5x
    if re.search(r'\.', formula) and not re.search(r'\d+\.\d[w-z]', formula): 
        formula = formula.split('.')
        for item in formula:
            prefactor = '1'
            for i in re.findall(p, item):
                pre, elem, num = i
                if pre:
                    prefactor = pre
                if num == '':
                    num = '1'
                num = prefactor + '*({})'.format(num)
                comp.append((elem, num))
    else:
        prefactor = '1'
        for i in re.findall(p, formula):
            pre, elem, num = i
            if pre:
                prefactor = pre
            if num == '':
                num = '1'
            num = prefactor + '*({})'.format(num)
            comp.append((elem, num))
    return comp 

def formula_reconstruct(composition, x=0.1, y=0.1, z=0.1, w=0.1):
    '''
    reconstruct chemical formula from composition
    composition in form of [(element,num), (element,num),...]
        element: string
        num: string, can be math expression such as '1+0.5x'

    return 
        flat chemcial formula: string, such as 'Ti1.5Cu0.1Au1.0'
    '''
    flat_list = []
    for (elem, num) in composition:
        num = re.sub(r'(\d)([w-z])', r'\1*\2', num) #convert 5x to 5*x
        flat_list.append(elem)
        flat_list.append(format(eval(num), '.1f'))
    return ''.join(flat_list)
  
def formula_parser(formula):
    return formula_reconstruct(formula_decompose(formula))

##  Manually Clean up Data Set

In [4]:
# problematic entries
print('1185: ' + df.loc[1185,'Density'])
print('2696: ' + df.loc[2696, "chemicalFormula"])
print('3892: ' + df.loc[3892,"chemicalFormula"])
print('3863: ' + df.loc[3863, "chemicalFormula"])

# need to modify parser to take care of leading parentheses
print('2902: ' + df.loc[2902, "chemicalFormula"] + ' ------leading parentheses cannot be parsed correctly yet')

1185: 3.16 (1%)
2696: Si3N4.x(BaOY2O3)
3892: B-N
3863: ^(13)C
2902: (3+x)Al2O3.(2-x)SiO2 ------leading parentheses cannot be parsed correctly yet


In [5]:
# update chemicalFormula by manually clean-up
df.loc[1185,"Density"] = '3.16'
df.loc[2696,"chemicalFormula"] = 'Si3N4.xBaOY2O3'
df.loc[3892,"chemicalFormula"] = 'BN'
df.loc[3863,"chemicalFormula"] = 'C'
# update chemicalFormula with cleanUp function
df["chemicalFormula"] = df["chemicalFormula"].map(cleanUp)

## DataFrame with new features

In [6]:
# create composition feature based on parsed formula
df["flatFormula"] = df["chemicalFormula"].map(formula_parser)
df["composition"] =df["flatFormula"].transform(str_to_composition)
df.shape

(4098, 163)

### add features based on composition (composition with floats)

In [7]:
# element property 
ep_feat = composition.ElementProperty.from_preset(preset_name="magpie")
df = ep_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True)# input the "composition" column to the featurizer
df.shape

(4098, 295)

In [8]:
# atomic orbitals
ao_feat = composition.AtomicOrbitals()
df = ao_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True)  
df.shape

(4098, 302)

In [9]:
# band center
bc_feat  = composition.BandCenter()
df = bc_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

(4098, 303)

In [10]:
# miedema
m_feat  = composition.Miedema()
df = m_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

(4098, 304)

In [11]:
# stoichiometry
s_feat  = composition.Stoichiometry()
df = s_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

(4098, 310)

In [12]:
# t metal fraction
tmf_feat  = composition.TMetalFraction()
df = tmf_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

(4098, 311)

In [13]:
# valence orbital
vo_feat  = composition.ValenceOrbital()
df = vo_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

(4098, 319)

In [14]:
# yang solid solution
yss_feat  = composition.YangSolidSolution()
df = yss_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

(4098, 321)

In [15]:
df.head()

,chemicalFormula,licenses,names,preparation,references,Axis Length,Axis Length-conditions,Axis Length-units,Bulk Modulus,Bulk Modulus-conditions,...,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,Yang omega,Yang delta
1,SiO2,[{'name': 'NIST_SRD-30'}],[Silica],NaN,"[{'citation': 'Gaseous Corrosion of Ceramics, ...",NaN,NaN,NaN,NaN,NaN,...,2.0,3.333333,0.000000,0.000000,0.375000,0.625000,0.000000,0.000000,NaN,0.273335
2,3Al2O3.2SiO2.xZrO2.yY2O3,[{'name': 'NIST_SRD-30'}],[Mullite composite],[{'name': 'Hot Pressing'}],[{'citation': 'Thermal Diffusivity/Conductivit...,NaN,NaN,NaN,NaN,NaN,...,2.0,2.935780,0.018349,0.000000,0.403704,0.592593,0.003704,0.000000,NaN,0.391165
3,Al2O3,[{'name': 'NIST_SRD-30'}],[Alumina],[{'name': 'Hot Pressing'}],"[{'citation': 'Mechanical Properties of Pure, ...",NaN,NaN,NaN,NaN,NaN,...,2.0,2.800000,0.000000,0.000000,0.416667,0.583333,0.000000,0.000000,NaN,0.397913
4,Si3N4.xCeO2.ySiC,[{'name': 'NIST_SRD-30'}],[Silicon nitride composite],[{'name': 'Hot Pressing'}],[{'citation': 'Effect of Silicon Carbide Whisk...,NaN,NaN,NaN,NaN,NaN,...,2.0,2.560000,0.013333,0.013333,0.436047,0.558140,0.002907,0.002907,NaN,0.243599
5,Si3N4,[{'name': 'NIST_SRD-30'}],[Silicon nitride],[{'name': 'Hot Isostatic Pressing'}],[{'citation': 'Corrosion of Silicon Nitride Ce...,NaN,NaN,NaN,NaN,NaN,...,2.0,2.571429,0.000000,0.000000,0.437500,0.562500,0.000000,0.000000,0.054313,0.213003


## the following three feature-generators also work; however...
- calculation of cohesive energy takes a long time
- calculation of atomic packing efficiency takes a  really really long time
- element fraction adds a lot of null columns. we might not need this

In [ ]:
# cohesive energy
# long calculation time
ce_feat  = composition.CohesiveEnergy()
df = ce_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

In [ ]:
# atomic packing efficiency
# very very long calculation time
ape_feat  = composition.AtomicPackingEfficiency()
df = ape_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape

In [ ]:
# element fraction
# It adds a lot null columns of elements
ef_feat  = composition.ElementFraction()
df = ef_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True) 
df.shape